In [1]:
import sqlite3

In [ ]:
#   1              2                   3
# User           DBMS               Database
# Python       Sqlite3   <--SQL--->   FILE
#     connection
#       cursor            CREATE, INSERT, SELECT

In [ ]:
con = sqlite3.connect('')
cur = con.cursor()

In [ ]:
# cur.execute       # SQL 1개 실행
# cur.executemany   # SQL 1개를 N번 반복 실행
# cur.executescript # (비표준) N개의 SQL문 실행

In [ ]:
cur.execute('''
  CREATE TABLE TEST (
    ID INTEGER,
    NAME CHAR(1)
  );
''')

In [ ]:
cur.execute('''
  INSERT INTO TEST (ID, NAME) VALUES (1, 'A');
''')

In [ ]:
cur.execute('SELECT * FROM TEST;')

In [ ]:
cur.execute('''
  INSERT INTO TEST (ID, NAME) VALUES (2, 'ABC');
''')

In [ ]:
# cur.fetchone   # 1행
# cur.fetchmany  # N행
# cur.fetchall   # 전체

In [ ]:
cur.fetchall()

[]

In [ ]:
cur.execute('INSERT INTO TEST VALUES (3,NULL);')

In [ ]:
cur.execute('SELECT * FROM TEST;') # 2
cur.fetchall()   # 1

[(1, 'A'), (2, 'ABC'), (3, None)]

In [ ]:
con.close()

콘솔을 닫고 다음 예제를 실행한다.

In [2]:
con = sqlite3.connect('')
cur = con.cursor()

In [3]:
con.execute('''
  CREATE TABLE CITY (
    CNO INTEGER PRIMARY KEY,
    NAME TEXT NOT NULL
  );
''')

In [4]:
con.execute('''
  INSERT INTO CITY (CNO, NAME) VALUES (1, '성북구');
''')

In [5]:
con.execute('''
  INSERT INTO CITY (NAME) VALUES ('강북구');
''')

In [6]:
con.execute('''
  INSERT INTO CITY VALUES (NULL, '동대문구');
''')

In [ ]:
con.execute('''
  INSERT INTO CITY VALUES (NULL, NULL);
''')
# 제약조건 위배로 인한 에러 발생

IntegrityError: NOT NULL constraint failed: CITY.NAME

In [ ]:
con.execute('''
  INSERT INTO CITY VALUES (3, '노원구');
''')
# CNO가 PK이므로 NOT NULL이면서 UNIQUE한 값이 아님.
# 앞선 구문에서 NULL을 넣었지만, 3이 자동으로 들어가면서

IntegrityError: UNIQUE constraint failed: CITY.CNO

In [7]:
cur.execute('SELECT * FROM CITY;')
cur.fetchall()

[(1, '성북구'), (2, '강북구'), (3, '동대문구')]

이를 통해서 SQLite에서의 Primary Key의 특징은 다음과 같다.

PrimaryKey -> NOT NULL, Unique, **Auto Increment**

In [8]:
cur.lastrowid

3

DB Browser for SQLite을 다운로드받아 로컬 데이터베이스의 상태를 확인할 수 있다. Table 구조 뜨는 거 확인하고, 이상한 점 찾기.

실컷 작업했는데 아무런 데이터가 안 들어가 있는 것을 확인할 수 있는데 이는 commit을 하지 않았기 빼문이다. 데이터의 수정을 반영하기 위해서는 반드시 commit이 필요하다.

    SQL
DBMS -> DB(?)
----    ----
Transaction
데이터가 추가/수정(갱신) -> 변화가 있을 때
Commit

In [9]:
con.commit()

In [10]:
help(con.executescript)

Help on built-in function executescript:

executescript(sql_script, /) method of sqlite3.Connection instance
    Executes multiple SQL statements at once.



In [11]:
# Parameter를 주는 방식 -> qmark
# c = [('중구',), ('강남구',), ('서초구',)]
con.execute('INSERT INTO CITY VALUES (NULL, ?);', ('중구',))

In [12]:
cur.execute('SELECT * FROM CITY;')
cur.fetchall()

[(1, '성북구'), (2, '강북구'), (3, '동대문구'), (4, '중구')]

In [13]:
# Parameter를 주는 방식 -> :named
con.execute('INSERT INTO CITY VALUES (NULL, :name);', {'name': '중랑구'})

In [14]:
cur.execute('SELECT * FROM CITY;')
cur.fetchall()

[(1, '성북구'), (2, '강북구'), (3, '동대문구'), (4, '중구'), (5, '중랑구')]

In [15]:
cur.executemany('''
  INSERT INTO CITY (NAME) VALUES (?)
''', (['노원구'], ['강남구'], ['서초구']))

# 여기까지 내용은 commit이 안 이루어졌기 때문에 3번까지밖에 들어가있지 않음.

In [16]:
type((['노원구'], ['강남구'], ['서초구'])), type((['노원구'], ['강남구'], ['서초구'])[0])

(tuple, list)

In [17]:
# exceutescript는 자동으로 커밋을 진행함.
cur.executescript('''
  DROP TABLE IF EXISTS SUPPLIER;
  CREATE TABLE SUPPLIER (
    SNO INTEGER PRIMARY KEY,
    NAME VARCHAR(2),
    CNO INTEGER INTEGER NOT NULL
  );

  DROP TABLE IF EXISTS PART;
  CREATE TABLE PART (
      PNO INTEGER PRIMARY KEY,
      NAME CHAR(1)
  );
''')

SQLite의 특징 중 하나인 **Serverless (Direct I/O)** -> FILE 1개만 Open 가능

connect 여러 번 하면 안 된다! Lock이 걸린다.

In [18]:
p = [{'n':'아메리카노'}, {'n':'카페라떼'}, {'n':'카페모카'}, {'n':'카푸치노'}, {'n':'프라페'}]
# len(p) -> 5, p[0] -> {'n':'아메리카노'}, p[0]['n] -> '아메리카노'

cur.executemany('INSERT INTO PART (NAME) VALUES (:n)', p) # 5번 반복, SQL문 1개, 각 반복마가 n키 값에 대응되는 value.

In [19]:
cur.execute('SELECT * FROM PART')
cur.fetchall()

[(1, '아메리카노'), (2, '카페라떼'), (3, '카페모카'), (4, '카푸치노'), (5, '프라페')]

cursor.execute 함수들이 하는 일

1. python이 파라미터 2개를 전달(SQL, 변수) <- 작업중
2. DBMS가 SQL을 해석
3. DB에 SQL을 적용

In [20]:
cur.execute('SELECT CNO FROM CITY WHERE NAME=?', ['성북구'])
cur.fetchone()

(1,)

In [21]:
cur.execute('SELECT CNO FROM CITY WHERE NAME LIKE ?', ['%구'])
cur.fetchall()

[(1,), (2,), (3,), (4,), (5,), (6,), (7,), (8,)]

In [22]:
cur.execute('''
    SELECT * FROM CITY WHERE NAME LIKE ?
    ORDER BY NAME
''', ['%구'])
cur.fetchall()

[(7, '강남구'),
 (2, '강북구'),
 (6, '노원구'),
 (3, '동대문구'),
 (8, '서초구'),
 (1, '성북구'),
 (4, '중구'),
 (5, '중랑구')]

In [23]:
# DBMS에서 슬라이싱을 맡기는 방법
# LIMIT 시작위치, 몇개
cur.execute('''
    SELECT * FROM CITY WHERE NAME LIKE ?
    ORDER BY NAME, CNO
    LIMIT 0, 1
''', ['%구'])

# cur.fetchmany(2) # n개, 슬라이싱의 주체는 파이썬
cur.fetchall()   # 전체

[(7, '강남구')]

In [24]:
# 게시판에서 검색어를 통해 글을 찾는 방법
# 정렬은 ORDER BY를 통햇 설정 가능
cur.execute('''
    SELECT * FROM CITY WHERE NAME LIKE ?
    ORDER BY NAME, CNO ASC
    LIMIT 0, 1
''', ['%성북구%'])

# Cursor -> DBMS <- DB
# cur.fetchone()    # record/tuple/row 단위로 가져옴 -> python iteratable 객체로 반환
# Cursor 결과 -> 읽어들임
# cur.fetchone()[0] 
# Cursor 결과 X
cur.fetchall()

[(1, '성북구')]

In [25]:
cur.execute('''
    SELECT * FROM CITY WHERE NAME LIKE ?
    ORDER BY NAME, CNO ASC
    LIMIT 0, 1
''', ['%성북구%'])
cno = cur.fetchone()[0] 


In [26]:
cur.execute('INSERT INTO SUPPLIER (NAME, CNO) VALUES(?, ?)', ('1호점', cno))
cur.execute('INSERT INTO SUPPLIER (NAME, CNO) VALUES(?, ?)', ('2호점', cno))
cur.execute('INSERT INTO SUPPLIER (NAME, CNO) VALUES(?, ?)', ('3호점', cno))

In [27]:
cur.execute('SELECT * FROM SUPPLIER')
cur.fetchall()

[(1, '1호점', 1), (2, '2호점', 1), (3, '3호점', 1)]

In [28]:
# 중첩 SQL
cur.execute('''
    INSERT INTO SUPPLIER (NAME, CNO) VALUES (?, (
        SELECT CNO FROM CITY WHERE NAME LIKE ?
        ORDER BY NAME, CNO ASC
        LIMIT 0, 1
    ))
''' , ('1호점', '%강북구%'))
    # INSERT INTO SUPPLIER (NAME, CNO) VALUES ('1호점', 2);
    # INSERT INTO SUPPLIER (NAME, CNO) VALUES ('1호점', 2 = (
    #     SELECT CNO FROM CITY WHERE NAME LIKE '%강북구%'
    #     ORDER BY NAME, CNO ASC
    #     LIMIT 0, 1
    # ))

In [29]:
cur.execute('SELECT * FROM SUPPLIER')
cur.fetchall()

[(1, '1호점', 1), (2, '2호점', 1), (3, '3호점', 1), (4, '1호점', 2)]

In [30]:
cur.execute('SELECT NAME FROM CITY WHERE CNO > 2')
c = cur.fetchall()

for name in c:
    name = '%'+name[0]+'%'

    # for no in ['1호점', '2호점', '3호점']:
    #     cur.execute('''
    #         INSERT INTO SUPPLIER (NAME, CNO) VALUES (?, (
    #             SELECT CNO FROM CITY WHERE NAME LIKE ?
    #             ORDER BY NAME, CNO ASC
    #             LIMIT 0, 1
    #         ))
    #     ''' , (no, name))
    params = [{'no':'1호점', 'name':name}, {'no':'2호점', 'name':name}, {'no':'3호점', 'name':name}]
    cur.executemany('''
        INSERT INTO SUPPLIER (NAME, CNO) VALUES (:no, (
            SELECT CNO FROM CITY WHERE NAME LIKE :name
            ORDER BY NAME, CNO ASC
            LIMIT 0, 1
        ))
    ''', params)

In [31]:
cur.execute('SELECT * FROM SUPPLIER')
len(cur.fetchall())

22

거래 이력 만들기

In [32]:
cur.execute('SELECT COUNT(PNO) FROM PART')
cur.fetchone()[0]

5

In [33]:
cur.execute('''
    CREATE TABLE SELLS(
        SNO INTEGER NOT NULL,
        PNO INTEGER NOT NULL,
        PRICE INTEGER NOT NULL DEFAULT 0
    )
''')

In [34]:
cur.executemany('INSERT INTO SELLS VALUES(?,?,?)',
[(1,1,10),
(2,1,10),
(3,1,10),
(5,1,10),
(7,1,10),
(9,1,10),
(11,1,10),
(1,2,15),
(3,2,15),
(5,2,15),
(8,2,15),
(2,3,12),
(3,3,12),
(4,3,12),
(6,3,12),
(12,3,12),
(10,4,8),
(13,4,8),
(16,4,8),
(19,4,8),
(22,4,8),
(21,5,20),
(18,5,20),
(15,5,20),
(12,5,20),
(9,5,20),
(6,5,20),
(3,5,20),]
)

In [35]:
con.commit()

In [36]:
cur.execute('SELECT NAME FROM CITY')
cur.fetchall()

[('성북구',),
 ('강북구',),
 ('동대문구',),
 ('중구',),
 ('중랑구',),
 ('노원구',),
 ('강남구',),
 ('서초구',)]

In [37]:
cur.execute('INSERT INTO CITY (NAME) VALUES("광진구");')

In [38]:
cur.execute('''
    SELECT CITY.NAME, SUPPLIER.NAME FROM CITY
    INNER JOIN SUPPLIER
    ON CITY.CNO=SUPPLIER.CNO
''')
len(cur.fetchall()) # 22개

22

In [39]:
cur.execute('''
    SELECT CITY.NAME, SUPPLIER.NAME FROM CITY
    LEFT JOIN SUPPLIER
    ON CITY.CNO=SUPPLIER.CNO
''')
len(cur.fetchall()) # 23개, [광진구, NULL]

23

In [40]:
cur.execute('''
    SELECT CITY.NAME, SUPPLIER.NAME FROM CITY
    LEFT JOIN SUPPLIER
    ON CITY.CNO=SUPPLIER.CNO
    GROUP BY CITY.CNO
    ORDER BY CITY.NAME ASC
''')
cur.fetchall()

[('강남구', '1호점'),
 ('강북구', '1호점'),
 ('광진구', None),
 ('노원구', '1호점'),
 ('동대문구', '1호점'),
 ('서초구', '1호점'),
 ('성북구', '1호점'),
 ('중구', '1호점'),
 ('중랑구', '1호점')]

In [41]:
cur.execute('''
    SELECT CITY.NAME, SUPPLIER.NAME, COUNT(SUPPLIER.NAME), SUM(SELLS.PRICE) FROM CITY
    LEFT JOIN SUPPLIER
    ON CITY.CNO=SUPPLIER.CNO
    INNER JOIN SELLS
    ON SUPPLIER.SNO=SELLS.SNO
    GROUP BY CITY.NAME, SUPPLIER.NAME
    ORDER BY CITY.CNO ASC
''')
cur.fetchall()

[('성북구', '1호점', 2, 25),
 ('성북구', '2호점', 2, 22),
 ('성북구', '3호점', 4, 57),
 ('강북구', '1호점', 1, 12),
 ('동대문구', '1호점', 2, 25),
 ('동대문구', '2호점', 2, 32),
 ('동대문구', '3호점', 1, 10),
 ('중구', '1호점', 1, 15),
 ('중구', '2호점', 2, 30),
 ('중구', '3호점', 1, 8),
 ('중랑구', '1호점', 1, 10),
 ('중랑구', '2호점', 2, 32),
 ('중랑구', '3호점', 1, 8),
 ('노원구', '2호점', 1, 20),
 ('노원구', '3호점', 1, 8),
 ('강남구', '2호점', 1, 20),
 ('강남구', '3호점', 1, 8),
 ('서초구', '2호점', 1, 20),
 ('서초구', '3호점', 1, 8)]

In [42]:
cur.execute('''
    SELECT CITY.NAME, SUPPLIER.NAME, COUNT(SUPPLIER.NAME), SUM(SELLS.PRICE) FROM CITY
    LEFT JOIN SUPPLIER
    ON CITY.CNO=SUPPLIER.CNO
    INNER JOIN SELLS
    ON SUPPLIER.SNO=SELLS.SNO
    GROUP BY CITY.NAME, SUPPLIER.NAME
    ORDER BY CITY.CNO ASC
''')
cur.fetchall()

[('성북구', '1호점', 2, 25),
 ('성북구', '2호점', 2, 22),
 ('성북구', '3호점', 4, 57),
 ('강북구', '1호점', 1, 12),
 ('동대문구', '1호점', 2, 25),
 ('동대문구', '2호점', 2, 32),
 ('동대문구', '3호점', 1, 10),
 ('중구', '1호점', 1, 15),
 ('중구', '2호점', 2, 30),
 ('중구', '3호점', 1, 8),
 ('중랑구', '1호점', 1, 10),
 ('중랑구', '2호점', 2, 32),
 ('중랑구', '3호점', 1, 8),
 ('노원구', '2호점', 1, 20),
 ('노원구', '3호점', 1, 8),
 ('강남구', '2호점', 1, 20),
 ('강남구', '3호점', 1, 8),
 ('서초구', '2호점', 1, 20),
 ('서초구', '3호점', 1, 8)]

In [ ]:
# SQL중첩 시킴(SQL안에 SQL) => Procedure
# JOIN(INNER=교집합, LEFT/RIGHT=기준테이블, OUTER)
# GROUP BY 

In [43]:
for d in con.iterdump():
    print(d)

BEGIN TRANSACTION;
CREATE TABLE CITY (
    CNO INTEGER PRIMARY KEY,
    NAME TEXT NOT NULL
  );
INSERT INTO "CITY" VALUES(1,'성북구');
INSERT INTO "CITY" VALUES(2,'강북구');
INSERT INTO "CITY" VALUES(3,'동대문구');
INSERT INTO "CITY" VALUES(4,'중구');
INSERT INTO "CITY" VALUES(5,'중랑구');
INSERT INTO "CITY" VALUES(6,'노원구');
INSERT INTO "CITY" VALUES(7,'강남구');
INSERT INTO "CITY" VALUES(8,'서초구');
INSERT INTO "CITY" VALUES(9,'광진구');
CREATE TABLE PART (
      PNO INTEGER PRIMARY KEY,
      NAME CHAR(1)
  );
INSERT INTO "PART" VALUES(1,'아메리카노');
INSERT INTO "PART" VALUES(2,'카페라떼');
INSERT INTO "PART" VALUES(3,'카페모카');
INSERT INTO "PART" VALUES(4,'카푸치노');
INSERT INTO "PART" VALUES(5,'프라페');
CREATE TABLE SELLS(
        SNO INTEGER NOT NULL,
        PNO INTEGER NOT NULL,
        PRICE INTEGER NOT NULL DEFAULT 0
    );
INSERT INTO "SELLS" VALUES(1,1,10);
INSERT INTO "SELLS" VALUES(2,1,10);
INSERT INTO "SELLS" VALUES(3,1,10);
INSERT INTO "SELLS" VALUES(5,1,10);
INSERT INTO "SELLS" VALUES(7,1,10);
INSERT INTO 